In [1]:
!pip install scrapy

In [0]:
import scrapy

In [0]:
class Spider12(scrapy.Spider):
  name = 'spider12'
  # Páginas que serán scrapeadas
  allowed_domains = ['pagina12.com.ar']
  custom_settings = {'FEED_FORMAT': 'json',
                     'FEED_URI': 'results.json',
                     'DEPTH_LIMIT': 2
                     }
  start_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                'https://www.pagina12.com.ar/secciones/universidad-diario',
                'https://www.pagina12.com.ar/secciones/economia',
                'https://www.pagina12.com.ar/secciones/sociedad',
                'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                'https://www.pagina12.com.ar/secciones/el-mundo',
                'https://www.pagina12.com.ar/secciones/deportes',
                'https://www.pagina12.com.ar/secciones/contratapa']
  
  def parse(self, response):
    # Main Article
    main_new = response.xpath('//div[@class="featured-article__container"]/h2/a/@href').get()
    if main_new is not None:
      yield response.follow(main_new, callback=self.parse_new)

    # Others news

    news = response.xpath('//ul[@class="article-list"]//li//a/@href').getall()
    for new in news:
      yield response.follow(new, callback=self.parse_new)
    
    # Next page
    next_page = response.xpath('a[@class="pagination-btn-next"]/@href')
    if next_page is not None:
      yield response.follow(next_page, callback=self.parse)
  

  def parse_new(self,  response):
    title = response.xpath('//h1[@class="article-title"]/text()').get()
    body = ''.join(response.xpath('//div[@class="article-text"]/p/text()').getall())
    yield {'url': response.url, 'title':title,'body':body}

In [0]:
from scrapy.crawler import CrawlerProcess

In [5]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()

2020-05-23 15:50:24 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-05-23 15:50:24 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-Ubuntu-18.04-bionic
2020-05-23 15:50:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-05-23 15:50:24 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 2}
2020-05-23 15:50:24 [scrapy.extensions.telnet] INFO: Telnet Password: b8c15379f82fd87f
/usr/local/lib/python3.6/dist-packages/scrapy/extensions/feedexport.py:210: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)
2020-05-23 15:50:24 [scrapy.middlew